# Code thu thập dữ liệu cho 2A và 2B
* Mục tiêu: Thu thập dữ liệu title, description của video để phân cụm chủ đề, categoryId (thể loại YouTube) để phân loại

## Import thư viện 

In [ ]:
import os
import csv
import json
import time
from googleapiclient.discovery import build
from tqdm import tqdm
import pandas as pd
import os
from dotenv import load_dotenv

## Load API Key Youtube

In [11]:
load_dotenv()  # load .env vào environment

API_KEY = os.getenv("YOUTUBE_API_KEY")
if not API_KEY:
    raise ValueError("YOUTUBE_API_KEY not found in environment")
# Khởi tạo service
youtube = build("youtube", "v3", developerKey=API_KEY)

## CẤU HÌNH 

### Hàm tìm video theo từ khóa
Hàm này tìm kiếm video trên YouTube theo từ khóa và trả về danh sách thông tin cơ bản của video (ID, tiêu đề, mô tả, ngày đăng, kênh):

In [12]:
# ==== Hàm tìm video theo từ khóa ====
def search_videos(query, max_results=30):
    request = youtube.search().list(
        q=query,
        part="id,snippet",
        type="video",
        maxResults=max_results
    )
    response = request.execute()

    videos = []
    for item in response.get("items", []):
        video_id = item["id"]["videoId"]
        title = item["snippet"]["title"]
        description = item["snippet"]["description"]
        published_at = item["snippet"]["publishedAt"]
        channel = item["snippet"]["channelTitle"]

        videos.append({
            "video_id": video_id,
            "title": title,
            "description": description,
            "published_at": published_at,
            "channel": channel
        })
    return videos



### Hàm lấy chi tiết video (thể loại, view, like, v.v.)
Hàm dưới đây lấy chi tiết video như thể loại, view, like, v.v...:

In [13]:
# ==== Hàm lấy chi tiết video (thể loại, view, like, v.v.) ====
def get_video_details(video_ids):
    request = youtube.videos().list(
        part="snippet,statistics",
        id=",".join(video_ids)
    )
    response = request.execute()

    details = []
    for item in response.get("items", []):
        video_id = item["id"]
        category_id = item["snippet"]["categoryId"]
        tags = item["snippet"].get("tags", [])
        view_count = item["statistics"].get("viewCount", 0)
        like_count = item["statistics"].get("likeCount", 0)
        comment_count = item["statistics"].get("commentCount", 0)

        details.append({
            "video_id": video_id,
            "category_id": category_id,
            "tags": tags,
            "view_count": view_count,
            "like_count": like_count,
            "comment_count": comment_count
        })
    return details

### Hàm thu thập dữ liệu
Tạo tạo List gồm danh sách 10 từ khóa có nội dung khác nhau, ta sẽ tìm video liên quan đến mỗi từ khóa để thu thập dữ liệu

Gọi hàm search_videos để tìm video theo từ khóa kw

Sau đó tạo list video_ids để gọi API chi tiết hơn

Gọi get_video_details để lấy thêm các thông tin thống kê

Ghép thông tin từ search_videos và get_video_details dựa trên video_id

Append toàn bộ video của một keyword vào list lớn all_videos

Chuyển all_videos (list dict) thành DataFrame Pandas, Lưu vào file youtube_videos.csv

In [ ]:
# ==== Main: Cào dữ liệu ====
keywords = ["Music", "News", "Sports", "Education", "Comedy", "Gaming", "Technology", "Travel", "Food", "Health"]

all_videos = []
for kw in keywords:
    videos = search_videos(kw, max_results=500)
    video_ids = [v["video_id"] for v in videos]

    # Lấy thêm chi tiết (category, views,...)
    details = get_video_details(video_ids)

    # Join 2 phần dữ liệu
    for v in videos:
        for d in details:
            if v["video_id"] == d["video_id"]:
                v.update(d)
    all_videos.extend(videos)

# Lưu ra CSV
df = pd.DataFrame(all_videos)
df.to_csv("Project_2_youtube_videos.csv", index=False, encoding="utf-8-sig")

print(" Đã lưu dữ liệu:", len(df), "video")

 Đã lưu dữ liệu: 500 video
